### ✅Step 1: Load and Explore AG News Dataset

In [1]:
import pandas as pd

Dataset's link -> https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

In [2]:
train_df = pd.read_csv(r"C:\Users\moham\OneDrive\Desktop\Courses\Elevvo Internship\News Category Classification\Data\train.csv")
test_df = pd.read_csv(r"C:\Users\moham\OneDrive\Desktop\Courses\Elevvo Internship\News Category Classification\Data\test.csv")

In [3]:
train_df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [4]:
train_df["Text"] = train_df["Title"].fillna('') + " " + train_df["Description"].fillna('')
test_df["Text"] = test_df["Title"].fillna('') + " " + test_df["Description"].fillna('')

In [5]:
train_df.head()

,Class Index,Title,Description,Text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...","Oil prices soar to all-time record, posing new..."


In [6]:
train_df['Class Index'].value_counts()

Class Index
3    30000
4    30000
2    30000
1    30000
Name: count, dtype: int64

### ✅ Step 2: Preprocess the Text

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [10]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)                  # remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)               # remove numbers and punctuation
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words]
    return " ".join(words)

In [11]:
train_df["Clean_Text"] = train_df["Text"].apply(preprocess)
test_df["Clean_Text"] = test_df["Text"].apply(preprocess)

In [12]:
train_df[["Text", "Clean_Text"]].head()

,Text,Clean_Text
0,Wall St. Bears Claw Back Into the Black (Reute...,wall st bear claw back black reuter reuter sho...
1,Carlyle Looks Toward Commercial Aerospace (Reu...,carlyl look toward commerci aerospac reuter re...
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,oil economi cloud stock outlook reuter reuter ...
3,Iraq Halts Oil Exports from Main Southern Pipe...,iraq halt oil export main southern pipelin reu...
4,"Oil prices soar to all-time record, posing new...",oil price soar alltim record pose new menac us...


### ✅ Step 3: TF-IDF Vectorization

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
# Use unigrams and bigrams
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))

In [15]:
X_train = vectorizer.fit_transform(train_df["Clean_Text"])
X_test = vectorizer.transform(test_df["Clean_Text"])

In [16]:
y_train = train_df["Class Index"]
y_test = test_df["Class Index"]

In [17]:
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (120000, 10000)
Test shape: (7600, 10000)
